In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time

In [2]:
url_part1 = 'http://www.stat-nba.com/query.php?page='
url_part3 = '&QueryType=all&AllType=season&AT=tot&crtcol=pts&order=1#label_show_result'
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'}
main_url = 'http://www.stat-nba.com'
score_info = []
k = 0

#初衷是计算1990-2019年这三十年，每年的TOP20得分榜，最保守估计需要20*30=600名球员信息，并且其排名是2019年的累计得分排名最靠前600的球员。
#实际上是完全不需要这么多球员信息，因为不少球员一旦在某一个年份进入历史的得分TOP20排行榜，之后的若干年也会存在于排行榜之中。
#实际上，跑取出600位球员数据后发现，在1990年累计得分第20位的球员是多尔夫-谢伊斯，他在2019年8月的累计得分榜上排名67位。由于每一页记录前20位球员信息，实际爬取4页就可以了。因
for i in range(4):
    
    #进入stat-nba网站中历史得分排行页面，通过循环进行翻页
    url = url_part1 + str(i) + url_part3  
    
    res = requests.get(url,headers=headers)

    res.encoding='utf-8'
    bs = BeautifulSoup(res.text,'html.parser')
    #获取当前页面所有球员的详情页tag对象
    player_urls = bs.find_all('a',class_='query_player_name')
    


    for play_url in player_urls:
        player_url_part = play_url['href'][1:]
        #获取每个球员的主页url
        player_url = main_url + player_url_part  
        
        res1 = requests.get(player_url,headers=headers)
        res1.encoding='utf-8'
        bs = BeautifulSoup(res1.text,'html.parser')
        
        #获取累计数据的父级tag对象
        sta_box_tot = bs.find('table',id = 'stat_box_tot')
        
        #获取数据标题信息，存放在列表中，用于后续定位得分列的索引。
        #因为少数球员由于年代久远，没有记录胜负信息，导致得分位于最后一列，而不是通常的倒数第三列。
        index_list = sta_box_tot.select('thead tr')[0].text.strip().split('\n')
        
        ser = pd.Series(index_list)
        
        #获取得分列在标题信息中的索引
        score_index = int(ser[ser.values == '得分'].index.values)
        
        
        #存储了该球员每个赛季的累计数据信息
        tag_1 = sta_box_tot.find_all('tr',class_='sort')
        
        #获取球员名称
        player_name = bs.find('div',class_='name').text.split('\n')[0] 
        
        #倒序获取数据，因为后续可视化部分需要年份从小往大。
        for tag in tag_1[::-1]:
            
            #获取原始格式的赛季‘88-89’
            season = tag.find_all('td')[1].text  
            
            #获取球队的信息。
            team = tag.find_all('td')[2].text 
            
            #将原始格式转换为4位的年份，方便后续统计。对于60年代的数据，会错误地转化为2060+，将在后续进行处理
            season = pd.to_datetime(str(season[-2:]),format='%y').year 
            
            #通过前面获取的的分裂的索引信息，来获取当前赛季的得分。因为第一个是空值，+1后才能正常获取到得分的信息。
            season_scores = tag.find_all('td')[score_index+1].text 
            
            
            #将每个球员每个赛季的得分情况放进列表中，作为后续DataFrame的值。
            score_info.append([player_name,season,team,season_scores]) 
            
        #如果球员在2019以前退役，那么补充退役年份到2019年的数据，方便后续统计指定年份的累计得分。
        while season < 2019:  
            season_scores = 0
            season += 1
            team = '无'
            score_info.append([player_name,season,team,season_scores])
            
        #方便观察当前的爬取进度
        k += 1
        
        print(k,player_name)
        #减少stat-nba服务器压力
        time.sleep(0.5)
        

        
frame = pd.DataFrame(np.array(score_info),columns=['球员','赛季','球队','得分'])
#将获取到的数据保存为excel
frame.to_excel(r'D:\数据分析\NBA球员每赛季得分.xlsx')

#提示爬取任务已完成
print('done')

1 卡里姆-贾巴尔/Kareem Abdul-Jabbar
2 卡尔-马龙/Karl Malone
3 科比-布莱恩特/Kobe Bryant
4 勒布朗-詹姆斯/LeBron James
5 迈克尔-乔丹/Michael Jordan
6 德克-诺维茨基/Dirk Nowitzki
7 威尔特-张伯伦/Wilt Chamberlain
8 沙奎尔-奥尼尔/Shaquille O'Neal
9 摩西-马龙/Moses Malone
10 埃尔文-海耶斯/Elvin Hayes
11 哈基姆-奥拉朱旺/Hakeem Olajuwon
12 奥斯卡-罗波特森/Oscar Robertson
13 多米尼克-威尔金斯/Dominique Wilkins
14 蒂姆-邓肯/Tim Duncan
15 保罗-皮尔斯/Paul Pierce
16 约翰-哈福利切克/John Havlicek
17 凯文-加内特/Kevin Garnett
18 阿历克斯-英格利什/Alex English
19 卡梅罗-安东尼/Carmelo Anthony
20 文斯-卡特/Vince Carter
21 雷吉-米勒/Reggie Miller
22 杰里-韦斯特/Jerry West
23 帕特里克-尤因/Patrick Ewing
24 雷-阿伦/Ray Allen
25 阿伦-艾弗森/Allen Iverson
26 查尔斯-巴克利/Charles Barkley
27 罗波特-帕里什/Robert Parish
28 阿德里安-丹特利/Adrian Dantley
29 德维恩-韦德/Dwyane Wade
30 埃尔金-贝勒/Elgin Baylor
31 凯文-杜兰特/Kevin Durant
32 克莱德-德雷克斯勒/Clyde Drexler
33 加里-佩顿/Gary Payton
34 拉里-伯德/Larry Bird
35 豪尔-格瑞尔/Hal Greer
36 沃尔特-贝拉米/Walt Bellamy
37 保罗-加索尔/Pau Gasol
38 鲍勃-佩蒂特/Bob Pettit
39 大卫-罗宾逊/David Robinson
40 乔治-格文/George Gervin
41 米奇-里奇蒙德/Mitch Richmond
42 乔-约翰逊/Joe Johnson

In [3]:
frame = pd.read_excel(r'D:\数据分析\NBA球员每赛季得分.xlsx')
frame.shape

(2892, 4)

In [4]:
#去掉ABA联盟球队的数据，因为这部分是不算做历史得分
frame.drop(frame[frame['球队'].str.contains('ABA')].index,inplace=True)
frame.shape

(2875, 4)

In [5]:
#去掉空值。由于有些球员数据距今太久远，没有记录其在某支球队的信息，只记录了其在该年份的总得分，因此可以直接去掉空值所在的行；
frame.dropna(inplace=True)
frame.shape

(2875, 4)

In [6]:
(frame['赛季'] > 2019).sum()

120

In [7]:
#将类似2065的数据转换成1965
def f(x):
    return (x-100) if x>2019 else x
frame['赛季'] = frame['赛季'].map(f)

In [8]:
(frame['赛季'] > 2019).sum()

0

In [9]:
#得到待填充的数据，因为有的球员在部分赛季没有打球，需要补充这部分数据，用于计算特定年份的累计得分信息。
#比如迈克尔乔丹，中途两次退役，如果不补充数据，那么这些年份的历史的分排行就不会有他，导致错误。
data_add = []

def f(frame):
    for i in range(frame.iloc[:,1].min() + 1,2019):
        if i not in frame.iloc[:,1].values:
            data_add.append([frame.iloc[0,0],i,'无',0])

frame.groupby('球员').apply(f)

""


In [10]:
frame_add = pd.DataFrame(np.array(data_add),columns=['球员','赛季','球队','得分'])
#将两部分数据进行拼接
frame = pd.concat([frame,frame_add],sort=False)
frame.shape

(3001, 4)

In [11]:
frame.index.duplicated().any()

True

In [12]:
frame.index = np.arange(3001)

In [13]:
frame.index.duplicated().any()

False

In [14]:
frame.duplicated(['球员','赛季']).sum()

88

In [15]:
#有的球员一个赛季在多个球队打球，并且该赛季有一个’合计‘值统计了该赛季的总分，因此需要删除之前的重复值，保留最后一个’合计‘值
frame.drop_duplicates(['球员','赛季'],keep='last',inplace=True)

In [16]:
frame.duplicated(['球员','赛季']).sum()

0

In [17]:
frame.shape

(2913, 4)

In [18]:
frame.dtypes

球员    object
赛季    object
球队    object
得分    object
dtype: object

In [19]:
frame['赛季'] = frame['赛季'].astype(int)
frame['得分'] = frame['得分'].astype(int)

In [20]:
frame1 = frame.copy()

In [21]:
frame1 = frame1.sort_values(['球员','赛季'],ascending=[True,True])
frame1.head()

,球员,赛季,球队,得分
1475,乔-约翰逊/Joe Johnson,2002,总计,581
1476,乔-约翰逊/Joe Johnson,2003,菲尼克斯太阳,803
1477,乔-约翰逊/Joe Johnson,2004,菲尼克斯太阳,1367
1478,乔-约翰逊/Joe Johnson,2005,菲尼克斯太阳,1400
1479,乔-约翰逊/Joe Johnson,2006,亚特兰大老鹰,1653


In [22]:
frame1['总得分'] = frame1.groupby('球员')['得分'].cumsum()

In [23]:
frame1.head()

,球员,赛季,球队,得分,总得分
1475,乔-约翰逊/Joe Johnson,2002,总计,581,581
1476,乔-约翰逊/Joe Johnson,2003,菲尼克斯太阳,803,1384
1477,乔-约翰逊/Joe Johnson,2004,菲尼克斯太阳,1367,2751
1478,乔-约翰逊/Joe Johnson,2005,菲尼克斯太阳,1400,4151
1479,乔-约翰逊/Joe Johnson,2006,亚特兰大老鹰,1653,5804


In [24]:
frame1.sort_values(['赛季','总得分'],ascending=[True,False],inplace=True)
frame1.head()

,球员,赛季,球队,得分,总得分
2309,多尔夫-谢伊斯/Dolph Schayes,1950,西拉丘斯国民,1072,1072
2310,多尔夫-谢伊斯/Dolph Schayes,1951,西拉丘斯国民,1121,2193
2311,多尔夫-谢伊斯/Dolph Schayes,1952,西拉丘斯国民,868,3061
2312,多尔夫-谢伊斯/Dolph Schayes,1953,西拉丘斯国民,1262,4323
2313,多尔夫-谢伊斯/Dolph Schayes,1954,西拉丘斯国民,1228,5551


In [25]:
(frame1['赛季'] < 1990).sum()

798

In [26]:
(frame1['赛季'] >= 1990).sum()

2115

In [27]:
frame1.drop(frame1[frame1['赛季'] < 1990].index,inplace=True)

In [28]:
frame1.shape

(2115, 5)

In [29]:
frame1.head()

,球员,赛季,球队,得分,总得分
20,卡里姆-贾巴尔/Kareem Abdul-Jabbar,1990,无,0,38387
207,威尔特-张伯伦/Wilt Chamberlain,1990,无,0,31419
332,埃尔文-海耶斯/Elvin Hayes,1990,无,0,27313
426,奥斯卡-罗波特森/Oscar Robertson,1990,无,0,26710
563,约翰-哈福利切克/John Havlicek,1990,无,0,26395


In [30]:
#取每一年历史的分前20的信息
def f(arr):
    return arr[:20]
frame1 = frame1.groupby('赛季').apply(f)

In [31]:
frame1.shape

(600, 5)

In [32]:
frame1.index = frame1.index.droplevel(0)
frame1.head()

,球员,赛季,球队,得分,总得分
20,卡里姆-贾巴尔/Kareem Abdul-Jabbar,1990,无,0,38387
207,威尔特-张伯伦/Wilt Chamberlain,1990,无,0,31419
332,埃尔文-海耶斯/Elvin Hayes,1990,无,0,27313
426,奥斯卡-罗波特森/Oscar Robertson,1990,无,0,26710
563,约翰-哈福利切克/John Havlicek,1990,无,0,26395


In [33]:
frame1.index = np.arange(600)

In [34]:
#只保留中文名
frame1['球员'] = frame1['球员'].map(lambda x:x.split('/')[0])

In [35]:
frame1.head()

,球员,赛季,球队,得分,总得分
0,卡里姆-贾巴尔,1990,无,0,38387
1,威尔特-张伯伦,1990,无,0,31419
2,埃尔文-海耶斯,1990,无,0,27313
3,奥斯卡-罗波特森,1990,无,0,26710
4,约翰-哈福利切克,1990,无,0,26395


In [36]:
#这两列的使命已经完成，可以丢掉
frame1.drop(['球队','得分'],axis=1,inplace=True)
frame1.head()

,球员,赛季,总得分
0,卡里姆-贾巴尔,1990,38387
1,威尔特-张伯伦,1990,31419
2,埃尔文-海耶斯,1990,27313
3,奥斯卡-罗波特森,1990,26710
4,约翰-哈福利切克,1990,26395


In [37]:
#可视化框架的格式要求
frame1['type'] = ''
frame1.head()

,球员,赛季,总得分,type
0,卡里姆-贾巴尔,1990,38387,
1,威尔特-张伯伦,1990,31419,
2,埃尔文-海耶斯,1990,27313,
3,奥斯卡-罗波特森,1990,26710,
4,约翰-哈福利切克,1990,26395,


In [38]:
frame1.rename(columns={'球员':'name','赛季':'date','总得分':'value'},inplace=True)
frame1.head()

,name,date,value,type
0,卡里姆-贾巴尔,1990,38387,
1,威尔特-张伯伦,1990,31419,
2,埃尔文-海耶斯,1990,27313,
3,奥斯卡-罗波特森,1990,26710,
4,约翰-哈福利切克,1990,26395,


In [39]:
frame1 = frame1.reindex(['name','type','value','date'],axis=1)
frame1.head()

,name,type,value,date
0,卡里姆-贾巴尔,,38387,1990
1,威尔特-张伯伦,,31419,1990
2,埃尔文-海耶斯,,27313,1990
3,奥斯卡-罗波特森,,26710,1990
4,约翰-哈福利切克,,26395,1990


In [40]:
frame1.to_csv(r'D:\数据分析\example.csv',index=False,encoding='UTF-8')
frame1.to_excel(r'D:\数据分析\example.xlsx',index=False,encoding='UTF-8')